In [93]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from keras.datasets import mnist
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

In [97]:
# split mnist dataset into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# splitting test set to testing and development set
X_test, X_dev, y_test, y_dev = train_test_split(X_test, 
                y_test, test_size=0.50, shuffle=True)

The train dataset is to train the model, the dev model is used to test the model and improve it if its under or overfitting, the test dataset is what we perform the final model on.

In [99]:
# resphaping datasets 
X_train = X_train.reshape((-1, X_train.shape[1]*X_train.shape[2]))
X_test = X_test.reshape((-1, X_test.shape[1]*X_test.shape[2]))
X_dev = X_dev.reshape((-1, X_dev.shape[1]*X_dev.shape[2]))

In [107]:
# seting the best score and random state
bscore = 0
bstate = 0
i = 1
# looping through first 13 random states and finding the best score from them
while i <=13:
    model = RandomForestClassifier(n_estimators=50, random_state=i)
    model.fit(X_train, y_train)
    nscore = model.score(X_dev, y_dev.flatten())
    if bscore < nscore:
        bscore = nscore
        bstate = i
    i+=1

I decided to tune random_state as it would take too long to loop through n_estimators

In [109]:
# returning the best score and the random state that got it
print(f'The best score was {bscore} with a random state of {bstate}')

The best score was 0.9654 with a random state of 6


In [111]:
# applying model on test data with random state with the best results
fmodel = RandomForestClassifier(n_estimators=100, random_state=bstate)
fmodel.fit(X_train, y_train)
print('Accuracy score on test data is', fmodel.score(X_test, y_test.flatten()))

Accuracy score on test data is 0.974


In [119]:
from sklearn.metrics import confusion_matrix

y_pred = fmodel.predict(X_test)

# creating and printing matrix as dataframe
classes = list([1,2,3,4,5,6,7,8,9,0])
conf_mat = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(conf_mat, columns=classes, index=classes)
cm_df

,1,2,3,4,5,6,7,8,9,0
1,469,0,1,0,0,2,1,1,2,0
2,0,576,2,2,0,0,0,0,0,0
3,1,0,498,2,1,0,2,5,4,0
4,0,0,6,500,0,3,0,3,3,1
5,1,0,0,0,474,0,4,0,3,6
6,0,1,0,6,0,452,2,1,1,0
7,5,1,0,0,0,1,472,0,1,0
8,0,2,8,0,0,0,0,476,2,2
9,2,0,3,5,2,2,2,1,468,4
0,4,2,0,4,4,1,0,1,4,485


In [122]:
# finding the hardest class

from sklearn.metrics import f1_score, precision_score, recall_score

# average f1 score
av_f1 = f1_score(y_test, y_pred, average='micro')
print(av_f1)

# f1 score per class
f = f1_score(y_test, y_pred, average=None)
lowest_score = min(f)
hardest_class = classes[list(f).index(lowest_score)]
print('Hardest class:', hardest_class)

0.974
Hardest class: 9


In [123]:
# precision and recall for 9  
prec = precision_score(y_test == classes.index(9), y_pred == classes.index(9))
rec = recall_score(y_test == classes.index(9), y_pred == classes.index(9))

print('Precision:', prec)
print('Recall:', rec)

Precision: 0.9590163934426229
Recall: 0.9570552147239264
